# wb_feature_selection_by_XGB_importance_scores
DC

#### This notebook:
* Load the EWAS aging data selecting for whole blood data
* Split data train/test 75/25
* Impute with mean of each train column
* Run XGboost 50 time on working data (took 4hrs)
* Rank cpgs by frequency in top 100 importance scores
* Save rankings as pickled 'cpgs_XGboost_blood_ranked'

#### Imports

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('poster')
import os

#### Import our modules

In [2]:
import age_predict.Loading_EWAS_Aging_Data as le
import age_predict.Regression as rg
import age_predict.Pickle_unpickle as pu

In [3]:
import os
# Get
os.getcwd()
#os.chdir('/Users/dcox/Dropbox')

'/Users/dcox/Desktop/EWAS-organized/blood/feature_selection'

In [4]:
# Paths
path_to_data = '../data/age_methylation_v1/'

#### set tissue of interest

In [5]:
tissue_of_interest = 'whole blood'

#### Load data

In [6]:
%%time
file_in = path_to_data + 'age_methylation_v1.txt'
file_out = path_to_data + tissue_of_interest + '_age_methylation_v1.txt'

df_t = le.load_EWAS_aging_by_tissue(file_in, file_out, search_term=tissue_of_interest,
                          num_rows='all', verbose=True)

finished line 10000
finished line 20000
finished line 30000
finished line 40000
finished line 50000
finished line 60000
finished line 70000
finished line 80000
finished line 90000
finished line 100000
finished line 110000
finished line 120000
finished line 130000
finished line 140000
finished line 150000
finished line 160000
finished line 170000
finished line 180000
finished line 190000
finished line 200000
finished line 210000
finished line 220000
finished line 230000
finished line 240000
finished line 250000
finished line 260000
finished line 270000
finished line 280000
finished line 290000
finished line 300000
finished line 310000
finished line 320000
finished line 330000
finished line 340000
finished line 350000
finished line 360000
finished line 370000
finished line 380000
finished line 390000
finished line 400000
finished line 410000
finished line 420000
finished line 430000
finished line 440000
finished line 450000
finished line 460000
finished line 470000
finished line 480000



#### Select age 20 and older

In [7]:
df_t_over20 = df_t[df_t.age >=20]

#### Split and impute data

In [8]:
%%time
df_p_working, df_p_saved = le.splitting_and_imputing(df_t_over20, input_percent=10, 
                                           fraction_test=0.25, seed=2021)

CPU times: user 3min 49s, sys: 35.5 s, total: 4min 24s
Wall time: 4min 36s


#### Save dataframes with columns from just ranked cpgs

In [10]:
# df_p_working_ranked = df_p_working[cpgs_XGboost_blood_ranked]
# df_p_working_ranked.insert(0, 'age', df_p_working.age)
# df_p_working_ranked.insert(0, 'tissue', df_p_working.tissue)
# df_p_saved_ranked = df_p_saved[cpgs_XGboost_blood_ranked]
# df_p_saved_ranked.insert(0, 'age', df_p_saved.age)
# df_p_saved_ranked.insert(0, 'tissue', df_p_saved.tissue)
# df_p_saved_ranked.to_csv('df_p_saved_ranked.csv')
# df_p_working_ranked.to_csv('df_p_working_ranked.csv')

#### Running XGboost 50 times to identify top 100 cpgs

In [11]:
%%time
# Runnin XGboost on working dataset 50 times with 50 different train/test splits
# This can take hours 
df_imp, rms_train, rms_test, r2_train, r2_test = rg.repeat_XGBoost(df_p_working, 50, test_size=0.20)

TypeError: repeat_XGBoost() got an unexpected keyword argument 'test_size'

In [ ]:
# write 50 trials results to a file, so don't have to rerun the 50 XGboost trials with the whole dataset again
file = 'df_importance_EWAS_age_' + tissue_of_interest + '_50_trials.csv'
#df_imp.to_csv(file)
#df_imp = pd.read_csv('./blood/df_importance_EWAS_age_whole_blood_50_trials.csv' , index_col=0)

In [ ]:
# Finding cpgs with on average the highest importance scores, see mean columnn in df below
df_imp_sorted = rg.importances_sorted_by_mean(df_imp)

In [ ]:
# This function makes a histogram of the frequency of cgs in the top 100 in the 50 trials above
inds, vs, h = rg.histogram_of_top_CpGs_by_importance(df_imp, 100)

In [ ]:
cpgs_XGboost_ranked_blood = h.index.values

In [ ]:
# Saving Rankings
# pu.pickle_object(cpgs_XGboost_ranked_blood, 'cpgs_XGboost_blood_ranked')